# Resources

* Aladdin Persson:  Variational Autoencoder from scratch in PyTorch

https://www.youtube.com/watch?v=VELQT1-hILo

https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/more_advanced/VAE


* Implement Deep Autoencoder in PyTorch for Image Reconstruction

Link: https://www.geeksforgeeks.org/implement-deep-autoencoder-in-pytorch-for-image-reconstruction/

* How to Generate Images using Autoencoders

Link: https://towardsdatascience.com/how-to-generate-images-using-autoencoders-acfbc6c3555e


https://medium.com/@rekalantar/variational-auto-encoder-vae-pytorch-tutorial-dce2d2fe0f5f

In [1]:
import torch
from torch import nn
from torchsummary import summary
import torchvision.datasets as datasets
import torch.optim as optim

import torchvision.transforms as transforms

C:\Users\latreche\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
other_data_dir = 'other_data'

In [39]:
batch_size = 32
workers = 4
shuffle = True

dataset = datasets.MNIST(root = other_data_dir, 
                         train = True, 
                         transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.5,), (0.5,)), # MNIST is grayscale
                             transforms.Lambda()
                         ]),
                         download=True)

dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size,
                                         shuffle = shuffle, num_workers = workers)

In [45]:
def _my_normalization(x):
    return (x + 1) / 2

In [22]:
a, b = next(iter(dataloader))

In [41]:
print(a.shape, torch.min(a), torch.max(a))

torch.Size([32, 1, 28, 28]) tensor(-1.) tensor(1.)


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_dim = 28*28
h_dim=200
z_dim=20

model = VariationalAutoEncoder(input_dim, h_dim, z_dim)
summary(model, (1, input_dim))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 200]         157,000
              ReLU-2               [-1, 1, 200]               0
            Linear-3                [-1, 1, 20]           4,020
            Linear-4                [-1, 1, 20]           4,020
            Linear-5               [-1, 1, 200]           4,200
              ReLU-6               [-1, 1, 200]               0
            Linear-7               [-1, 1, 784]         157,584
Total params: 326,824
Trainable params: 326,824
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.25
Estimated Total Size (MB): 1.26
----------------------------------------------------------------


In [42]:
learning_rate = 0.0003

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.BCELoss(reduction = "sum")

In [43]:
n_epochs = 10

In [45]:
model.train()
for epoch in range(n_epochs):
    overall_loss = 0
    for batch_idx, (x, _) in enumerate(dataloader):
        x = x.view(batch_size, input_dim).to(device)
        print(torch.max(x), torch.min(x))
        optimizer.zero_grad()

        x_reconstructed, mu, sigma = model(x)

        print(torch.max(x_reconstructed), torch.min(x_reconstructed))
        #loss = criterion(x, x_reconstructed, mean, log_var) for kl divergence
        #loss = criterion(x, x_reconstructed)
        loss = ((x - x_hat)**2).sum()
        
        #overall_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss/(batch_idx*batch_size))

tensor(1.) tensor(-1.)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(1.) tensor(-1.)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [6]:
import torch
from torch import nn


class VariationalAutoEncoder(nn.Module):
    def __init__(self, input_dim, h_dim=200, z_dim=20):
        super().__init__()
        # encoder
        self.img_2hid = nn.Linear(input_dim, h_dim)
        self.hid_2mu = nn.Linear(h_dim, z_dim)
        self.hid_2sigma = nn.Linear(h_dim, z_dim)

        # decoder
        self.z_2hid = nn.Linear(z_dim, h_dim)
        self.hid_2img = nn.Linear(h_dim, input_dim)

        self.relu = nn.ReLU()

    def encode(self, x):
        h = self.relu(self.img_2hid(x))
        mu, sigma = self.hid_2mu(h), self.hid_2sigma(h)
        return mu, sigma

    def decode(self, z):
        h = self.relu(self.z_2hid(z))
        return torch.sigmoid(self.hid_2img(h))

    def forward(self, x):
        mu, sigma = self.encode(x)
        epsilon = torch.randn_like(sigma)
        z_new = mu + sigma*epsilon
        x_reconstructed = self.decode(z_new)
        return x_reconstructed, mu, sigma